In [1]:
from sentence_transformers import SentenceTransformer
import os
from qdrant_client import QdrantClient, models
import numpy
from typing import List, Dict, Any
import uuid
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("Import Successful")

Import Successful


In [2]:
load_dotenv()

True

In [3]:
def setup_qdrant_client():
    """Initialize the Qdrant client with credentials from .env file"""
    return QdrantClient(
        url=os.environ.get("QDRANT_URL"), 
        api_key=os.environ.get("QDRANT_API_KEY"),
    )

In [8]:
qdrant_client = setup_qdrant_client()

In [4]:
def get_query_embedding(query: str):
    return embedding_model.encode(query).tolist()

def retrieve_relevant_chunks(query: str, top_k: int = 5):
    query_vector = get_query_embedding(query)

    search_result = qdrant_client.query_points(
        collection_name="ragchatbot_standard",
        query=query_vector,
        limit=5,
        with_payload=True
    )

    all_chunk_texts = []
    for _, scored_points in search_result:
        for point in scored_points:
            chunk_text = point.payload.get("chunk_text", "")
            all_chunk_texts.append(chunk_text)
    
    # Remove duplicates while preserving order
    unique_chunk_texts = list(dict.fromkeys(all_chunk_texts))
    
    # Join into a context string
    context = "\n\n".join(unique_chunk_texts)
    return context

def build_prompt(context, question):
    return f"""You are a helpful assistant. Answer the question using only the context provided.

Context:
{context}

Question:
{question}
"""

def generate_answer(prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32)
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

In [6]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [9]:
query = "tell me about marriott"


context = retrieve_relevant_chunks(query)
prompt = build_prompt(context, query)
answer = generate_answer(prompt)

print("🔍 Answer:\n", answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔍 Answer:
 You are a helpful assistant. Answer the question using only the context provided.

Context:
Neom, named after “New Future”, is a large-scale futuristic urban development project with US$500 billion in funding from Saudi Arabia’s sovereign wealth fund and will be located in the northwest of the Kingdom of Saudi Arabia (NEOM, n.d. a). Plans for four distinct regions of Neom have been officially revealed thus far with hints and speculation of more to come (Corder, 2023a; Ravenscroft, 2023). Among the four regions is Sindalah, a luxury resort artificial island in the Red Sea featuring attractions such as yachting, golf, luxury retail outlets and is planned to be open in 2024 (Arab News, 2023; NEOM, n.d. b). Marriott International (Marriott), an international powerhouse in the hotel industry with 31 brands and over 8500 properties across the planet, has signed an agreement with NEOM to open 3 hotels in Sindalah (Mitchell, 2023; Tapley, 2023; Arabian Business, 2023). This paper wi